# Convert Genesis DFL to Wavefront to SRWLWfr

This example will show how a DFL file produced by Genesis can be converted to an opemPMD-wavefront by lume-genesis, and then converted to an SRW wavefront object.

# Get Genesis v2 DFL file and write Wavefront

In [1]:
# Get example DFL file
!wget -N https://portal.nersc.gov/cfs/m669/wavefronts/hxr_cut9_5keVTaper4_5.out.dfl

--2020-09-02 14:06:16--  https://portal.nersc.gov/cfs/m669/wavefronts/hxr_cut9_5keVTaper4_5.out.dfl
Resolving portal.nersc.gov (portal.nersc.gov)... 128.55.206.28, 128.55.206.24, 128.55.206.26
Connecting to portal.nersc.gov (portal.nersc.gov)|128.55.206.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 501991968 (479M)
Saving to: ‘hxr_cut9_5keVTaper4_5.out.dfl’

hxr_cut9_5keVTaper4 100%[===================>] 478.74M  25.2MB/s    in 26s     

2020-09-02 14:06:42 (18.4 MB/s) - ‘hxr_cut9_5keVTaper4_5.out.dfl’ saved [501991968/501991968]



In [2]:
from h5py import File
from genesis.writers import write_openpmd_wavefront_h5

In [3]:
DFL = 'hxr_cut9_5keVTaper4_5.out.dfl'

# Genesis parameters. These need to be known to populate the openPMD-wavefront metadata
PARAM={'ncar': 251, 'dgrid': 0.00015, 'xlamds': 1.5005e-10, 'zsep': 20, 'ntail':0, 'itdp':1}

In [4]:
from genesis.parsers import parse_genesis_dfl

In [5]:
%%time
D2 = parse_genesis_dfl(DFL, nx=PARAM['ncar'])

CPU times: user 717 ms, sys: 481 ms, total: 1.2 s
Wall time: 637 ms


In [6]:
PARAM['xlamds']*PARAM['zsep']

3.001e-09

In [7]:
with File('wavefront.h5', 'w') as h5:
    write_openpmd_wavefront_h5(h5, dfl=D2, param=PARAM)

In [8]:
h5 = File('wavefront.h5', 'r')
dict(h5['data']['000000']['meshes']['electricField'].attrs)

{'axisLabels': array(['x', 'y', 'z'], dtype=object),
 'geometry': 'cartesian',
 'gridGlobalOffset': array([-1.50000e-04, -1.50000e-04, -7.47249e-07]),
 'gridSpacing': array([1.200e-06, 1.200e-06, 3.001e-09]),
 'gridUnitDimension': array([[1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0]]),
 'gridUnitSI': array([1., 1., 1.]),
 'photonEnergy': 8262.85894256583,
 'photonEnergyUnitDimension': array([ 2,  1, -2,  0,  0,  0,  0]),
 'photonEnergyUnitSI': 1.602176634e-19,
 'timeOffset': 0.0,
 'unitDimension': array([ 1,  1, -3, -1,  0,  0,  0])}

# Wavefront to SRW

In [9]:
from pmd_wavefront.interfaces.srw import srw_wfr_from_openpmd_wavefront

?srw_wfr_from_openpmd_wavefront

Signature:
srw_wfr_from_openpmd_wavefront(
    h5_meshes,
    dtype=dtype('float32'),
    iz_start=None,
    iz_end=None,
    iz_step=None,
    SRWLWfr_class=None,
)
Docstring:
Forms the essential input for SRW's wfr class from an open handle to openPMD-wavefront meshes.

Optionally iz_start, iz_end, and iz_step can be given which appropriately sample the full data in the z dimension. 

SRWLWfr_class should be the SRWLWfr class. If this is not given, then the raw inputs needed will be returned:
    arEx, arEy, kwargs,  wrf_attrs
And then the wavefront can be constructed with:
    wfr = SRWLWfr_class(arEx, arEy, **kwargs)
    wfr.__dict__.update(wrf_attrs)        
    
Note that the units in an openPMD-wavefront electric field E are V/m, and SRW's field F_SRW are sqrt(W/mm^2):
    E = F_SRW * sqrt(2*Z_0) * 1000 mm/m
    

Example with SRW:
    h5 = File('wavefront.h5', 'r')
    meshes = h5['data']['000000']['meshes']
    
    from ?? import SRWLWfr
    
    wrf = srw_wfr_from_openpmd_wa

In [10]:
with File('wavefront.h5', 'r') as h5:
    arEx, arEy, kwargs,  wrf_attrs = srw_wfr_from_openpmd_wavefront(h5['data']['000000']['meshes'],  iz_step=None)

In [11]:
# These are the SRW arrays
len(arEx), len(arEy)

(62748996, 62748996)

In [12]:
# kwargs and attributes for the SRWLWfr class
kwargs,  wrf_attrs

({'_typeE': 'f',
  '_eStart': 0,
  '_eFin': 4.975098472957582e-15,
  '_ne': 498,
  '_xStart': -0.00015,
  '_xFin': 0.00015,
  '_nx': 251,
  '_yStart': -0.00015,
  '_yFin': 0.00015,
  '_ny': 251,
  '_zStart': 0},
 {'avgPhotEn': 8262.85894256583, 'presFT': 1, 'unitElFld': 2})

# Cleanup

In [13]:
import os
os.remove('hxr_cut9_5keVTaper4_5.out.dfl')
os.remove('wavefront.h5')